In [220]:
import flow
import grpc
import json
import csv

In [212]:
nodes_list = [{'node':'access-001.mainnet1.nodes.onflow.org:9000', 'start_at': 7601063, 'end_at': 8742958},
              {'node':'access-001.mainnet2.nodes.onflow.org:9000', 'start_at': 8742959, 'end_at': 9737132},
              {'node':'access-001.mainnet3.nodes.onflow.org:9000', 'start_at': 9737133, 'end_at': 9992019},
              {'node':'access-001.mainnet4.nodes.onflow.org:9000', 'start_at': 9992020, 'end_at': 12020336},
              {'node':'access.mainnet.nodes.onflow.org:9000', 'start_at': 12020336, 'end_at': 12706374}
             ]

def get_purchases(node, start, delta):
    
    channel = grpc.insecure_channel(node)
    stub = flow.access.access_pb2_grpc.AccessAPIStub(channel)
    response = stub.GetEventsForHeightRange(flow.access.access_pb2.GetEventsForHeightRangeRequest(
        type='A.c1e4f4f4c4257510.Market.MomentPurchased',
        start_height = start,
        end_height = start + delta))

    return response

In [ ]:
csv_file = csv.writer(open("MomentPurchased.csv", "w+"))

csv_file.writerow(['block_height', 'block_timestamp','event_id', 'moment_id', 'sell_price', 'address'])

for node in nodes_list:
    for Block_height in range(node['start_at'], node['end_at'] - (node['end_at'] % 100), 100):
        purch = get_purchases(node['node'], Block_height, 100)
        for _, block in enumerate(purch.results):
            block_height = block.block_height
            block_timestamp = float(str(block.block_timestamp.seconds) + '.' + str(block.block_timestamp.nanos))
            for _, event in enumerate(block.events):
                try:
                    event_transaction_id = event.transaction_id
                    json_payload = json.loads(event.payload)

                    csv_file.writerow([block_height, block_timestamp, json_payload["value"]["id"], 
                                  int(json_payload["value"]["fields"][0]["value"]["value"]), 
                                  float(json_payload["value"]["fields"][1]["value"]["value"]), 
                                  json_payload["value"]["fields"][2]["value"]["value"]["value"]])
                except:
                    pass

In [136]:
js = json.loads(purch.results[0].events[1].payload)

In [164]:
js

{'type': 'Event',
 'value': {'id': 'A.c1e4f4f4c4257510.Market.MomentPurchased',
  'fields': [{'name': 'id', 'value': {'type': 'UInt64', 'value': '5102947'}},
   {'name': 'price', 'value': {'type': 'UFix64', 'value': '60.00000000'}},
   {'name': 'seller',
    'value': {'type': 'Optional',
     'value': {'type': 'Address', 'value': '0xa67c296b3b9c00c2'}}}]}}